# sqlite3 In Python


In [8]:
import sqlite3

In [9]:
type(sqlite3)

module

In [10]:
conn = sqlite3.connect("first.db")

In [11]:
in_mem = sqlite3.connect(":memory:")

# DBAPI 2.0


# Connections And Cursors


In [13]:
conn

In [26]:
cur = conn.cursor()

In [25]:
# control structure
# traversal of
# records in a db
# = cursor


In [28]:
cur.execute("""
    CREATE TABLE ice_cream_flavors (
        id INTEGER PRIMARY KEY,
        flavor TEXT,
        rating INTEGER
    );
""")


OperationalError: table ice_cream_flavors already exists

# Inserts


In [30]:
cur.execute("""
    INSERT INTO ice_cream_flavors (flavor, rating)
    VALUES ('choco', 10), ('straw', 9), ('pistachio', 7.2);
""")

In [31]:
conn.commit()


# BONUS: DML vs DDL (and DCL and DQL)


In [ ]:
cur.execute("INSERT INTO ice_cream_flavors (flavor, rating) "
            "VALUES ('chocolate', 10), ('strawberry', 9), ('pistachio', 7.2);")

# ^needed conn.commit()

In [ ]:
cur.execute('CREATE TABLE ice_cream_flavors (id INTEGER PRIMARY KEY, flavor TEXT, rating INTEGER);')

# ^did not need conn.commit()

In [32]:
# DML - insert, update, delete
# DDL - create/alter/drop table
# DCL - grant, revoke
# DQL - select, with


# Fetching


In [33]:
# > fetchone()
# > fetchmany()
# > fetchall()

In [38]:
cur.execute("SELECT * FROM ice_cream_flavors;")


In [39]:
cur.fetchone()

(1, 'choco', 10)

In [40]:
cur.fetchone()

(2, 'straw', 9)

In [42]:
next(cur)

StopIteration: 

In [44]:
cur.fetchone()

In [45]:
cur.execute("SELECT * FROM ice_cream_flavors;")

for row in cur:
    print(row)

(1, 'choco', 10)
(2, 'straw', 9)
(3, 'pistachio', 7.2)


In [46]:
# lazy eval


In [47]:
cur.execute("SELECT * FROM ice_cream_flavors;")

cur.fetchall()


[(1, 'choco', 10), (2, 'straw', 9), (3, 'pistachio', 7.2)]

In [48]:
cur.execute("SELECT * FROM ice_cream_flavors;")

cur.fetchmany(2)

[(1, 'choco', 10), (2, 'straw', 9)]

In [49]:
cur.fetchmany(2)


[(3, 'pistachio', 7.2)]

In [ ]:
# fetchone() -> fetchmany(1)
# fetchall() -> fetchmany(n), n = nr records in the result set

# Column Names Would Be Nice


In [53]:
cur.execute('SELECT * FROM ice_cream_flavors;')

cur.fetchall()

[(1, 'choco', 10), (2, 'straw', 9), (3, 'pistachio', 7.2)]

In [54]:
# row['flavor'] -> straw


In [55]:
cur.execute('SELECT * FROM ice_cream_flavors;')

result = cur.fetchone()

result['flavor']

TypeError: tuple indices must be integers or slices, not str

In [58]:
cur.row_factory = None

In [60]:
# - list of column names
# - seq of records

In [61]:
cur.description


(('id', None, None, None, None, None, None),
 ('flavor', None, None, None, None, None, None),
 ('rating', None, None, None, None, None, None))

In [62]:
cur.row_factory = lambda cursor, row: dict(zip([col[0] for col in cursor.description], row))


In [63]:
cur.execute('SELECT * FROM ice_cream_flavors;')

result = cur.fetchone()

result['flavor']


'choco'

In [64]:
result['rating']


10

In [65]:
result


{'id': 1, 'flavor': 'choco', 'rating': 10}

In [66]:
type(result)


dict

In [68]:
cur.execute('SELECT * FROM ice_cream_flavors;')

result = cur.fetchall()

result

[{'id': 1, 'flavor': 'choco', 'rating': 10},
 {'id': 2, 'flavor': 'straw', 'rating': 9},
 {'id': 3, 'flavor': 'pistachio', 'rating': 7.2}]

In [69]:
sqlite3.Row


sqlite3.Row

In [70]:
cur.row_factory = None


In [71]:
cur.execute('SELECT * FROM ice_cream_flavors;')

result = cur.fetchall()

result


[(1, 'choco', 10), (2, 'straw', 9), (3, 'pistachio', 7.2)]

In [72]:
cur.row_factory = sqlite3.Row


In [73]:
cur.execute('SELECT * FROM ice_cream_flavors;')

result = cur.fetchall()

result

In [74]:
first_row = result[0]


In [75]:
first_row['flavor']


'choco'

In [76]:
first_row['rating']

10

In [77]:
first_row['id']


1

In [79]:
first_row[1]

'choco'

In [80]:
type(first_row)


sqlite3.Row

In [82]:
cur.execute("INSERT INTO ice_cream_flavors (flavor, rating) VALUES ('hazelnut', 6);")

In [85]:
# conn.commit()

In [ ]:
conn.rollback()

In [ ]:
# BEGIN
#     * WORK *
# COMMIT | ROLLBACK

# Parameterized Statements


In [ ]:
cur.execute("INSERT INTO ice_cream_flavors (flavor, rating) VALUES ('hazelnut', 6);")
conn.commit()


In [86]:
flavor = input("Enter a flavor: ")
rating = input("Enter a rating: ")

cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES ('{flavor}', {rating});")
conn.commit()


In [87]:
# cur.executescript()


In [ ]:
flavor = "chocolate', 1); DROP TABLE ice_cream_flavors; -- "

In [88]:
flavor = input("Enter a flavor: ")
rating = input("Enter a rating: ")

cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
            (flavor, rating))
conn.commit()

# A Different Flavor


In [89]:
flavor = input("Enter a flavor: ")
rating = input("Enter a rating: ")

cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (:flavor, :rating);",
            {
                "flavor": flavor,
                "rating": rating
            })
conn.commit()


In [ ]:
# sqlite3
# positional placeholder    -> ?
# named placeholders        -> :name


# Various Ways To Execute


In [90]:
cur.execute(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
            ('fudge', 6.7))
conn.commit()

In [91]:
# .executemany()


In [92]:
ice_cream_flavors = [
    ('vanilla', 4.5),
    ('strawberry', 9),
    ('mint', 3.0),
    ('coffee', 6.0),
    ('caramel', 8),
    ('saltedcaramel', 9.2),
]


In [93]:
cur.executemany(f"INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
            ice_cream_flavors)


In [94]:
conn.commit()


In [ ]:
# .executescript()

In [95]:
# - create new toppings table
# - seed it with some values


In [97]:
cur.executescript("""
CREATE TABLE IF NOT EXISTS toppings (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL
);

INSERT INTO toppings (name) VALUES ('sprinkles'), ('choco sauce'), ('caramel sauce'),
('nuts');
""")

In [98]:
conn.commit()


# A Convenient Shortcut


In [99]:
conn.execute("INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
            ('mediterranean', 6.7))

conn.commit()


In [100]:
cur = conn.execute("SELECT * FROM ice_cream_flavors;")


In [101]:
cur


In [102]:
cur.fetchall()


[(1, 'choco', 10),
 (2, 'straw', 9),
 (3, 'pistachio', 7.2),
 (4, 'hazelnut', 6.3),
 (5, 'cherries', 4.3),
 (6, 'banana', 2.1),
 (7, 'fudge', 6.7),
 (8, 'vanilla', 4.5),
 (9, 'strawberry', 9),
 (10, 'mint', 3),
 (11, 'coffee', 6),
 (12, 'caramel', 8),
 (13, 'saltedcaramel', 9.2),
 (14, 'mediterranean', 6.7)]

# BONUS: Database Dump


In [106]:
obj = conn.iterdump()

In [109]:
next(obj)


'INSERT INTO "ice_cream_flavors" VALUES(1,\'choco\',10);'

In [110]:
for line in conn.iterdump():
    print(line)


BEGIN TRANSACTION;
CREATE TABLE ice_cream_flavors (
        id INTEGER PRIMARY KEY,
        flavor TEXT,
        rating INTEGER
    );
INSERT INTO "ice_cream_flavors" VALUES(1,'choco',10);
INSERT INTO "ice_cream_flavors" VALUES(2,'straw',9);
INSERT INTO "ice_cream_flavors" VALUES(3,'pistachio',7.2);
INSERT INTO "ice_cream_flavors" VALUES(4,'hazelnut',6.3);
INSERT INTO "ice_cream_flavors" VALUES(5,'cherries',4.3);
INSERT INTO "ice_cream_flavors" VALUES(6,'banana',2.1);
INSERT INTO "ice_cream_flavors" VALUES(7,'fudge',6.7);
INSERT INTO "ice_cream_flavors" VALUES(8,'vanilla',4.5);
INSERT INTO "ice_cream_flavors" VALUES(9,'strawberry',9);
INSERT INTO "ice_cream_flavors" VALUES(10,'mint',3);
INSERT INTO "ice_cream_flavors" VALUES(11,'coffee',6);
INSERT INTO "ice_cream_flavors" VALUES(12,'caramel',8);
INSERT INTO "ice_cream_flavors" VALUES(13,'saltedcaramel',9.2);
INSERT INTO "ice_cream_flavors" VALUES(14,'mediterranean',6.7);
CREATE TABLE toppings (
    id INTEGER PRIMARY KEY,
    name TEXT

In [111]:
_file = open("ice_cream_db_dump.sql", "w")

for line in conn.iterdump():
    _file.write(f"{line}\n")

_file.close()


In [112]:
with open("ice_cream_db_dump2.sql", "w") as _file:
    for line in conn.iterdump():
        _file.write(f"{line}\n")


# Context Managers


In [ ]:
conn.execute("INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
             ('super_salted_caramel', 2.7))
conn.commit()

In [113]:
with conn:
    conn.execute("INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?);",
             ('super_salted_caramel', 2.7))
    # because ^ success, conn.commit() was called

In [114]:
with conn:
    conn.execute("INSERT INTO ice_cream_flavors (flavor, rating) VALUES (?, ?, ?);",
             (15, 'super_salted_caramel', 2.7))
    # because ^ failed, conn.rollback() was called

OperationalError: 3 values for 2 columns

In [117]:
conn.execute("SELECT * FROM ice_cream_flavors LIMIT 3;").fetchall()


ProgrammingError: Cannot operate on a closed database.

In [116]:
conn.close()
